## Accessing the project directory on my Google Drive

In [ ]:
import os
from google.colab import drive

drive.mount('/drive', force_remount=False)

Mounted at /drive


In [ ]:
project_dir = "/drive/My Drive/RNN_seq2seq"

# change working directory to project_dir
os.chdir(project_dir)

## Dependencies

In [ ]:
import random
import pandas as pd
from string import ascii_lowercase
from IPython.display import clear_output

from os import makedirs
from os.path import join

import torch.nn as nn
import torch.optim as optim

from scripts.utils import read_datasets, save_dict_as_json
from scripts.dataloader import get_text_encoder_decoder, customize_dataloader_func
from scripts.visualization import plot_training_log, plot_performances_per_seq_len
from scripts.pytorch_utils import get_model, count_parameters, train_and_evaluate, evaluate, get_results

## Log time and random seeds

In [ ]:
from datetime import datetime
import random
import numpy as np
import torch

now = datetime.now()
print("Time stamp:", now.strftime("%Y-%m-%d %H:%M:%S"))

seed = 3266473
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

Time stamp: 2023-05-30 15:22:45


## Data loading and processing

In [ ]:
train, dev, test, gen = read_datasets("data/identity")


def quadratic_copying(w):
    return w * len(w)

def modify_dataset(ds):

    out = []
    for t, _ in ds:
        out.append([t, quadratic_copying(t)])

    return out

train = modify_dataset(train)
dev = modify_dataset(dev)
test = modify_dataset(test)
gen = modify_dataset(gen)

In [ ]:
len(train), len(dev), len(test), len(gen)

(10000, 10000, 50000, 100000)

In [ ]:
in_vocab = ascii_lowercase
out_vocab = ascii_lowercase
in_seq_encoder, in_seq_decoder = get_text_encoder_decoder(in_vocab)
out_seq_encoder, out_seq_decoder = get_text_encoder_decoder(out_vocab)

In [ ]:
dataloader_func = customize_dataloader_func(in_seq_encoder,
                                            out_seq_encoder,
                                            padding_idx=1,
                                            batch_size=1000,
                                            shuffle=False)

train_dl = dataloader_func(train)
dev_dl = dataloader_func(dev)
test_dl = dataloader_func(test, batch_size=5000)
gen_dl = dataloader_func(gen, batch_size=5000)

## Experiments

In [ ]:
task_name = "quadratic_copying_attn_srnn_model_complexity"

metadata = []
main_results = []
task_folder = join("experiments", task_name)
meta_col = ["RNN", "Attention", "lr", "Hidden size", "Embedding size", "Param #", "Max Epoch #"]
main_res_col = ["Run #", "RNN", "Attention", "Dataset", "Loss",
                "Full Sequence Accuracy", "First N-symbol Accuracy", "Overlap Rate"]


for rnn_type in ["SRNN"]:
    for use_attn in [True]:
        for run_num in range(1, 4):

            print(f"RNN type: {rnn_type}; use attention: {use_attn}; run num: {run_num}\n")

            embd_dim = 384
            hidden_size = 640
            learning_rate = 5e-4
            weight_decay = 1e-5
            max_epoch_num = 500

            ModelConfig = {"rnn_type": rnn_type,
                           "embd_dim": embd_dim,
                           "hidden_size": hidden_size,
                           "device": "cuda",
                           "num_layers": 1,
                           "dropout_rate": 0.0,
                           "use_attention": use_attn,
                           "bidirectional": False,
                           "in_vocab_size": len(in_vocab) + 2, # for the two boundary symbols
                           "out_vocab_size": len(out_vocab) + 2, # for the two boundary symbols
                           "reduction_method": "sum"}

            model = get_model(ModelConfig)

            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

            attn = "attn" if use_attn else "attn-less"
            folder = join(task_folder, rnn_type, attn, str(run_num))
            makedirs(folder, exist_ok=True)

            saved_model_fp = join(folder, "model.pt")
            log = train_and_evaluate(model, train_dl, dev_dl,
                                     criterion, optimizer,
                                     saved_model_fp=saved_model_fp,
                                     acc_threshold=-0.1,
                                     print_eval_freq=10,
                                     max_epoch_num=max_epoch_num,
                                     train_exit_acc=0.9999,
                                     eval_exit_acc=0.995,
                                     teacher_forcing_ratio=1.0)

            save_dict_as_json(log, join(folder, "training_log.json"))
            save_dict_as_json(ModelConfig, join(folder, "ModelConfig.json"))
            plot_training_log(log, show_plot=False, saved_plot_fp=join(folder, "training_plot.png"))

            train_res, dev_res = get_results(log, train_log=True)

            _, perfs_train = evaluate(model, train_dl, criterion,
                                      per_seq_len_performance=True)
            _, perfs_dev = evaluate(model, dev_dl, criterion,
                                      per_seq_len_performance=True)

            test_aggr, perfs_test = evaluate(model, test_dl, criterion,
                                             per_seq_len_performance=True)

            gen_aggr, perfs_gen = evaluate(model, gen_dl, criterion,
                                             per_seq_len_performance=True)

            test_res = get_results(test_aggr, train_log=False)
            gen_res = get_results(gen_aggr, train_log=False)

            for res, ds in zip([train_res, dev_res, test_res, gen_res], ["Train", "Dev", "Test", "Gen"]):
                main_results.append([run_num, rnn_type, use_attn, ds] + res)


            save_dict_as_json(perfs_train, join(folder, "perfs_train.json"))
            save_dict_as_json(perfs_dev, join(folder, "perfs_dev.json"))
            save_dict_as_json(perfs_test, join(folder, "perfs_test.json"))
            save_dict_as_json(perfs_gen, join(folder, "perfs_gen.json"))
            plot_performances_per_seq_len(perfs_train, False, join(folder, "perfs_train.png"))
            plot_performances_per_seq_len(perfs_dev, False, join(folder, "perfs_dev.png"))
            plot_performances_per_seq_len(perfs_test, False, join(folder, "perfs_test.png"))
            plot_performances_per_seq_len(perfs_gen, False, join(folder, "perfs_gen.png"))

            clear_output(wait=True)

        param_num = count_parameters(model)
        metadata.append([rnn_type, use_attn, learning_rate, hidden_size, embd_dim, param_num, max_epoch_num])

pd.DataFrame(metadata, columns=meta_col).to_csv(join(task_folder, "metadata.csv"), index=False)
pd.DataFrame(main_results, columns=main_res_col).to_csv(join(task_folder, "main_results.csv"), index=False)

RNN type: SRNN; use attention: True; run num: 3

The model has 2,582,812 trainable parameters
Current epoch: 10, 
training performance: {'loss': 2.9027841329574584, 'full sequence accuracy': 0.0, 'first n-symbol accuracy': 0.005936748275012066, 'overlap rate': 0.15338684017145435}
evaluation performance: {'loss': 2.9118266105651855, 'full sequence accuracy': 0.0, 'first n-symbol accuracy': 0.0053571025004998055, 'overlap rate': 0.15329330692989737}

Current epoch: 20, 
training performance: {'loss': 3.508083462715149, 'full sequence accuracy': 0.0, 'first n-symbol accuracy': 0.047054632797225854, 'overlap rate': 0.25122152999305547}
evaluation performance: {'loss': 3.5694653511047365, 'full sequence accuracy': 0.0, 'first n-symbol accuracy': 0.04146041570488951, 'overlap rate': 0.23906317741110872}

Current epoch: 30, 
training performance: {'loss': 0.6595512151718139, 'full sequence accuracy': 0.0002, 'first n-symbol accuracy': 0.8779052664558821, 'overlap rate': 0.9216610869872668}
e

## Automatically disconnect and delete the runtime

In [ ]:
from google.colab import runtime
runtime.unassign()